In [ ]:
# !pip install pyspark

     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     -------------------------------------- 0.0/317.3 MB 487.6 kB/s eta 0:10:51
     ---------------------------------------- 0.2/317.3 MB 1.3 MB/s eta 0:04:04
     ---------------------------------------- 0.4/317.3 MB 2.4 MB/s eta 0:02:11
     ---------------------------------------- 0.8/317.3 MB 3.7 MB/s eta 0:01:25
     ---------------------------------------- 1.4/317.3 MB 5.4 MB/s eta 0:00:58
     ---------------------------------------- 2.1/317.3 MB 6.6 MB/s eta 0:00:48
     ---------------------------------------- 2.9/317.3 MB 7.9 MB/s eta 0:00:40
     ---------------------------------------- 3.6/317.3 MB 8.9 MB/s eta 0:00:36
      --------------------------------------- 4.3/317.3 MB 9.2 MB/s eta 0:00:35
      -------------------------------------- 5.1/317.3 MB 10.1 MB/s eta 0:00:31
      -------------------------------------- 5.6/317.3

In [11]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Create SparkSession
spark = SparkSession.builder.appName('Welmart Sales Insights').getOrCreate()

# Create DataFrame
csv_file = "Superstore.csv"
df = spark.read.format("csv").option("header", "true").load(csv_file)


In [10]:
display(df.show())

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

None

In [14]:
df.createOrReplaceTempView('superstore')

# 1. Determine the best-selling product sub-category.

In [28]:
sales_per_subcategory = df.groupBy('Sub-Category').agg(F.sum('Sales').alias('Total Sales')).orderBy('Total Sales', ascending=False).withColumn('Total Sales', F.round('Total Sales', 3))
sales_per_subcategory.show()

+------------+-----------+
|Sub-Category|Total Sales|
+------------+-----------+
|      Phones| 329753.088|
|      Chairs| 328449.103|
|     Storage| 216803.212|
|      Tables| 206965.532|
|     Binders| 199905.717|
|    Machines| 189238.631|
| Accessories| 167380.318|
|     Copiers|  149528.03|
|   Bookcases| 114879.996|
|  Appliances| 107532.161|
| Furnishings|   82752.23|
|       Paper|  75356.118|
|    Supplies|   45952.47|
|         Art|  27118.792|
|   Envelopes|   15339.49|
|      Labels|  12486.312|
|   Fasteners|   3008.656|
+------------+-----------+



In [27]:
# Round to 3 decimal places
sales_per_subcategory.limit(1).show()

+------------+-----------+
|Sub-Category|Total Sales|
+------------+-----------+
|      Phones| 329753.088|
+------------+-----------+



# 2. Identify the product category generating the highest revenue.

In [29]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [34]:
revenue_per_category = df.groupBy('Category').agg(F.sum('Profit').alias('Total Profit')).orderBy('Total Profit', ascending=False).withColumn('Total Profit', F.round('Total Profit', 3))
revenue_per_category.show()

+---------------+------------+
|       Category|Total Profit|
+---------------+------------+
|     Technology|  145388.297|
|Office Supplies|  120632.878|
|      Furniture|   19686.427|
+---------------+------------+



In [31]:
revenue_per_category.limit(1).show()

+----------+------------+
|  Category|Total Profit|
+----------+------------+
|Technology|  145388.297|
+----------+------------+



# 3. Compile a top 10 list of the most valuable customers.

In [35]:
top10_valuable_customer = df.groupBy('Customer ID').agg(F.sum('Profit').alias('Total Profit')).orderBy('Total Profit', ascending=False).withColumn('Total Profit', F.round('Total Profit', 3)).limit(10)
top10_valuable_customer.show()

+-----------+------------+
|Customer ID|Total Profit|
+-----------+------------+
|   TC-20980|    8964.483|
|   RB-19360|    6976.096|
|   SC-20095|    5757.412|
|   HL-15040|    5622.429|
|   AB-10105|    5438.908|
|   TA-21385|    4703.788|
|   CM-12385|     3899.89|
|   KD-16495|    3038.625|
|   AR-10540|    2884.621|
|   DR-12940|    2869.076|
+-----------+------------+



# 4. Determine the state responsible for the highest number of orders.

In [36]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [37]:
orders_by_state = df.groupBy('State').agg(F.sum('Quantity').alias('Total Quantity')).orderBy('Total Quantity', ascending=False).withColumn('Total Quantity', F.round('Total Quantity', 3))
orders_by_state.show()

+--------------+--------------+
|         State|Total Quantity|
+--------------+--------------+
|    California|     13637.272|
|      New York|      5116.198|
|         Texas|      4272.414|
|  Pennsylvania|      3614.383|
|    Washington|      3541.886|
|      Illinois|      2903.226|
|          Ohio|      2863.187|
|       Florida|      1939.531|
|    New Jersey|       1602.74|
|       Arizona|      1596.944|
|      Colorado|        1465.0|
|      Michigan|       1196.03|
|      Virginia|       1192.63|
|North Carolina|       1011.68|
|       Georgia|        895.16|
|     Tennessee|       891.103|
|      Missouri|        888.24|
|      Oklahoma|        784.24|
|      Delaware|         768.9|
| Massachusetts|        684.23|
+--------------+--------------+
only showing top 20 rows



In [38]:
orders_by_state.limit(1).show()

+----------+--------------+
|     State|Total Quantity|
+----------+--------------+
|California|     13637.272|
+----------+--------------+



# 5. Find the year with the highest revenue generation.

In [39]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [53]:
# create new column Year, that corresponds to last 4 characters of Order Date
df_with_year = df.withColumn('Year', F.substring('Order Date', -4, 4))
df_with_year.show()


+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Year|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+----+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-

In [54]:
revenue_by_year = df_with_year.groupBy('Year').agg(F.sum('Profit').alias('Total Profit')).orderBy('Total Profit', ascending=False).withColumn('Total Profit', F.round('Total Profit', 3))
revenue_by_year.show()

+----+------------+
|Year|Total Profit|
+----+------------+
|2017|   93642.646|
|2016|   81247.208|
|2015|   61756.922|
|2014|   49060.826|
+----+------------+

